In [1]:
import os
from glob import glob
import numpy as np
import dask
import xarray as xr
import cartopy.crs as ccrs
from cmocean import cm
import xgcm

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)
#print(cluster.job_script())
w = cluster.start_workers(30)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [15]:
client

Client Scheduler: tcp://10.148.1.24:42032 Dashboard: http://10.148.1.24:8787/status,Cluster Workers: 60 Cores: 240 Memory: 3000.00 GB


_________
# Read dataset


In [14]:
## open files, nc files  ##
ncdata_dir = '/home1/scratch/xyu/mit_nc_new/'

# number of fies to load for each face (1 file = 60 days)
Nf = 1
face_60 = [1]

v_u = 'SSU'
v_v = 'SSV'
v_eta = 'Eta'


In [6]:
## Here we do DFT on u+iv ##

for face in face_60:
    
    # SSU #
    filenames_u = sorted(glob(ncdata_dir+v_u+'_f%02d_t*.nc' %(face)))
    filenames_u = filenames_u[:Nf]
    ds_u = xr.open_mfdataset(filenames_u, chunks={'time': 1}, concat_dim='time', compat='equals')
    
    # SSV #
    filenames_v = sorted(glob(ncdata_dir+v_v+'_f%02d_t*.nc' %(face)))
    filenames_v = filenames_v[:Nf]    
    ds_v = xr.open_mfdataset(filenames_v, chunks={'time': 1}, concat_dim='time', compat='equals')

    # Eta#
    filenames_Eta = sorted(glob(ncdata_dir+v_eta+'_f%02d_t*.nc' %(face)))
    filenames_Eta = filenames_Eta[:Nf]        
    ds_Eta = xr.open_mfdataset(filenames_Eta, chunks={'time': 1}, concat_dim='time', compat='equals')
    
    # grid file
    grid = xr.open_dataset('/home1/scratch/xyu/mit_grid_t.nc')
    grid = grid.isel(face=face_60)

In [7]:
# Merge into one xarray dataset #
ds = xr.merge([ds_u,ds_v,ds_Eta,grid])
print(ds)

<xarray.Dataset>
Dimensions:  (face: 1, i: 4320, i_g: 4320, j: 4320, j_g: 4320, time: 1440)
Coordinates:
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * face     (face) int64 1
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 5.713e+06 5.717e+06 ...
    dtime    (time) datetime64[ns] dask.array<shape=(1440,), chunksize=(1,)>
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
    XC       (face, i, j) float32 ...
    YC       (face, i, j) float32 ...
    XG       (face, i_g, j_g) float32 ...
    YG       (face, i_g, j_g) float32 ...
    rA       (face, i, j) float32 ...
    dxG      (face, i, j_g) float32 ...
    dyG      (face, j, i_g) float32 ...
    Depth    (face, i, j) float32 ...
    rAz      (face, i_g, j_g) float32 ...
    dxC      (face, j, i_g) float

In [8]:
time_begin = 1
time_end = 119

In [16]:
grid = xgcm.Grid(ds, periodic=['X', 'Y'])
grid

<xgcm.Grid>
X Axis (periodic):
  * center   i (4320) --> left
  * left     i_g (4320) --> center
Y Axis (periodic):
  * center   j (4320) --> left
  * left     j_g (4320) --> center

---


# 1. Time derivative of SSV

In [9]:
SSV_forward = ds['SSV'].isel(time=slice(time_begin+1,time_end+1))
SSV_forward = SSV_forward.drop(['dtime','time'])
print(SSV_forward)

<xarray.DataArray 'SSV' (i: 4320, j_g: 4320, time: 118)>
dask.array<shape=(4320, 4320, 118), dtype=float32, chunksize=(4320, 4320, 1)>
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
Dimensions without coordinates: time


In [10]:
SSV_backward = ds['SSV'].isel(time=slice(time_begin-1,time_end-1))
SSV_backward = SSV_backward.drop(['dtime','time'])
print(SSV_backward)

<xarray.DataArray 'SSV' (i: 4320, j_g: 4320, time: 118)>
dask.array<shape=(4320, 4320, 118), dtype=float32, chunksize=(4320, 4320, 1)>
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
Dimensions without coordinates: time


In [12]:
d_SSV_dt_center_face2 = ((SSV_forward - SSV_backward)/2/3600).load()

KeyboardInterrupt: 

# 2. Gradient of KE
Finally, we calculate the kinetic energy $1/2 (u^2 + v^2)$ by interpolating both quantities into the cell center point (i.e. (i, j)).

In [17]:
KE = (grid.interp(ds['SSU']**2,'X') + grid.interp(ds['SSV']**2,'Y'))/2
print(KE)

<xarray.DataArray (i: 4320, j: 4320, time: 1440)>
dask.array<shape=(4320, 4320, 1440), dtype=float32, chunksize=(4319, 4319, 1)>
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * time     (time) float64 5.702e+06 5.706e+06 5.71e+06 5.713e+06 5.717e+06 ...


In [21]:
dKE_dy = ((grid.diff(KE,'Y', boundary='extend'))/ds.dyC).load()
print(dKE_dy)

KeyboardInterrupt: 

In [23]:
cluster.scheduler.close()

<Future finished result=None>